# Pair Programming ETL Transformación III

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [479]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ast

## Cargamos el `csv` en el que se trabajará

In [480]:
df = pd.read_csv("files/1.paises_meteo_info.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [481]:
df2 = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col =0)
df2.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [482]:
df.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_type', 'prec_amount', 'snow_depth',
       'wind10m.direction', 'wind10m.speed', 'pais'],
      dtype='object')

## Definición de variables

In [483]:
# Definimos las variables que vamos a necesitar para hacer las 
# llamadas a cada uno de los métodos de la clase.

dic_paises = {"USA":[-100.445882,39.7837304,], 
            "Australia":[134.755,-24.7761086],
            "South_Africa":[24.991639,-28.8166236],
            "New Zealand":[172.8344077,-41.5000831],
            "Papua New Guinea":[144.2489081,-5.6816069]}
producto_meteo = "meteo"
df_empty = pd.DataFrame()
name_file_wind = "3.clima.csv"
name_file_rh = "files/4.clima_wind.csv"

## Creamos la clase 

In [534]:
class Extraccion: 
    # definimos el método constructor con las variables globales que usaremos.
    def __init__(self, diccionario_paises, dataframe_ataques): # Recibe un diccionario con los países y coordenadas, el dataframe del archivo araques de tiburones y en formato string el nombre de la columna "layer"
        
        self.diccionario_paises = diccionario_paises
        self.dataframe_ataques = dataframe_ataques
        self.columna = "layer"
        
    # El método llamará a la API para cada país deseado

    def llamada_api (self, dataframe_vacio, producto): # recibe un dataframe vacío y el nombre del producto a solicitar a la api

        # Definimos la variable 
        self.dataframe_vacio = dataframe_vacio
        self.producto = producto

        # hacemos la llamada  a la API iterando por cada uno de los países
        for k,v in self.diccionario_paises.items():
            lon = (v[0])
            lat = (v[1])

            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        
        response = requests.get(url=url)
        
        codigo_estado = response.status_code
        
        razon_estado = response.reason
        
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        
        response.json()
        # Se convierten los datos obtenidos en un df
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        
        # Creamos una columna llamada país donde se vaya registrando cada país
        df["country"] = k

        # Concatenamos los df que hemos obtenido con cada país
        df_final = pd.concat([self.dataframe_vacio,df],axis=0, ignore_index= True)

        # Se guarda el df obtenido con la información de la API
        df_final.to_csv("files/3.1.clima.csv")

        return df_final

    # Este método filtra el df sobre ataques de tiburones a los países deseados
    def filtrar_por_paises (self): #no recibe parámetro
        
        df_ataques_tib = self.dataframe_ataques[(self.dataframe_ataques["country"]== 'usa') | 
            (self.dataframe_ataques["country"]== "australia") | 
            (self.dataframe_ataques["country"]== "new zealand") |
            (self.dataframe_ataques["country"]== "south africa") |
            (self.dataframe_ataques["country"]== "papua new guinea")]
    
        return df_ataques_tib

    # Este método limpia la columna 'wind_profile' del dataframe obtenido a través de la API
    def limpiar_columnas_wind (self, nombre_archivo_wind):

        self.nombre_archivo_wind = nombre_archivo_wind

        # Se abre el archivo que contiene la información obtenida de la API
        df3 = pd.read_csv(f"files/{self.nombre_archivo_wind}")

        df3.reset_index()

        df3['wind_profile']= df3['wind_profile'].apply(ast.literal_eval)

        z = df3['wind_profile'].apply(pd.Series)
        z.head()

        # Por eso empezamos con un for para iterar por cada una de las columnas. 
        for i in range(len(z.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_dir_" + str(z[i].apply(pd.Series)[self.columna][0]) 

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(z[i].apply(pd.Series)['direction'])

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df3.insert(i, nombre, valores)

        df3.to_csv("files/4.clima_wind.csv")

        return df3

    # Este método limpia la columna 'rh_profile' del dataframe obtenido a través de la API
    def limpiar_columnas_rh (self,nombre_archivo_rh):
        
        self.nombre_archivo_rh = nombre_archivo_rh

        # Se abre el archivo creado con la limpieza de la columna wind_profile
        df4 = pd.read_csv(self.nombre_archivo_rh)

        df4.reset_index()

        df4['rh_profile']= df4['rh_profile'].apply(ast.literal_eval)

        z = df4['rh_profile'].apply(pd.Series)
        z.head()

        # Itera por cada una de las columnas. 
        for i in range(len(z.columns)): 

            # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(z[i].apply(pd.Series)[self.columna][0]) 

            # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(z[i].apply(pd.Series)["rh"] )

            # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df4.insert(i, nombre, valores)

            # Guardamos el dataframe obtenido con la limpieza de las columnas.
            df4.to_csv("files/5.clima_wind_rh.csv")

        return df4

    

## Aplicación de la clase Extracción

In [535]:
# Creamos una variable llamada api que es resultado de la aplicación de la 
# clase Extracción, recibe como argumentos latitud, longitud y ciudad (Variables definidas anteriormente).

api = Extraccion(dic_paises,df2)
api

In [536]:
df_country = api.llamada_api (df_empty, producto_meteo)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [537]:
df_country.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [538]:
df2.columns

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud', 'country2', 'fatal_N', 'fatal_Unknown',
       'fatal_Y', 'fatal_N.1', 'fatal_Unknown.1', 'fatal_Y.1', 'species_.1',
       'fecha_limpia.1', 'type.1', 'age_NORM'],
      dtype='object')

In [559]:
df_ataques_filtrado = api.filtrar_por_paises()

In [560]:
df_ataques_filtrado.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [541]:
df_wind_limpio = api.limpiar_columnas_wind (name_file_wind)

In [542]:
df_wind_limpio.head(2)

,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,100,90,85,80,80,85,75,20,295,300,...,25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,100,95,85,80,75,80,70,345,280,290,...,26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [543]:
df_rh_limpio = api.limpiar_columnas_rh (name_file_rh)

In [544]:
df_rh_limpio.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,14,10,7,8,6,-1,-3,-2,-4,-4,...,25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,14,14,7,8,5,-1,-3,-3,-4,-4,...,26,2,11,1013,none,1,0,100,3,Papua New Guinea


## Creación de Funciones

Este paso ha sideo realizado para comprobar que los futuros métodos de la clase funcionan

In [545]:
# Este método llamará a la API para cada país

def seleccionar_paises (diccionario_paises, producto):

    # Definimos la variable producto
    df_final = pd.DataFrame()

    # hacemos la llamada  a la API iterando por cada uno de los países
    for k,v in diccionario_paises.items():
        lon = (v[0])
        lat = (v[1])

        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
    
    response = requests.get(url=url)
    
    codigo_estado = response.status_code
    
    razon_estado = response.reason
    
    if codigo_estado == 200:
        print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 402:
        print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    elif codigo_estado == 404:
        print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    else:
        print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
    
    response.json()
    # Se convierten los datos obtenidos en un df
    df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    
    # Creamos una columna llamada país donde se vaya registrando cada país
    df["pais"] = k

    # Concatenamos los df que hemos obtenido con cada país
    df_final = pd.concat([df_final,df],axis=0, ignore_index= True)

    df_final.to_csv("files/3.clima.csv")

    return df_final


In [546]:
df3 = seleccionar_paises(dic_paises,producto_meteo)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [547]:
df3.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


Método filtrar  

In [548]:
df_ataques = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col = 0)
df_ataques.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [549]:
def filtrar_por_paises (dataframe1):

    df_atakks = dataframe1[(dataframe1["country"]== 'usa') | 
        (dataframe1["country"]== "australia") | 
        (dataframe1["country"]== "new zealand") |
        (dataframe1["country"]== "south africa") |
        (dataframe1["country"]== "papua new guinea")]
    
    return df_atakks

In [550]:
df_filtrado = filtrar_por_paises(df_ataques)
df_filtrado.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [551]:
df_filtrado["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

Método limpiar wind

In [552]:
df3.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [553]:
def limpiar_columnas_wind (nombre_archivo):

    df4 = pd.read_csv(f"files/{nombre_archivo}")

    df4.reset_index()

    df4['wind_profile']= df4['wind_profile'].apply(ast.literal_eval)

    z = df4['wind_profile'].apply(pd.Series)
    z.head()

    # Por eso empezamos con un for para iterar por cada una de las columnas. 
    for i in range(len(z.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
        nombre = "wind_dir_" + str(z[i].apply(pd.Series)["layer"][0]) 

        # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
        valores = list(z[i].apply(pd.Series)["direction"])

        # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
        df4.insert(i, nombre, valores)

    df4.to_csv("files/4.clima_wind.csv")

    return df4


In [554]:
df_nuevo= limpiar_columnas_wind ("3.clima.csv")
df_nuevo.head(2)

,wind_dir_950mb,wind_dir_900mb,wind_dir_850mb,wind_dir_800mb,wind_dir_750mb,wind_dir_700mb,wind_dir_650mb,wind_dir_600mb,wind_dir_550mb,wind_dir_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,100,90,85,80,80,85,75,20,295,300,...,25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,100,95,85,80,75,80,70,345,280,290,...,26,2,11,1013,none,1,0,100,3,Papua New Guinea


In [555]:
#df4.reset_index()

#df4['wind_profile']= df4['wind_profile'].apply(ast.literal_eval)

#z = df3['wind_profile'].apply(pd.Series)
#z.head()

Limpiar columnas rh

In [556]:
def limpiar_columna_rh (nombre_archivo):

    df4 = pd.read_csv(nombre_archivo)

    df4.reset_index()

    df4['rh_profile']= df4['rh_profile'].apply(ast.literal_eval)

    z = df4['rh_profile'].apply(pd.Series)
    z.head()

    # Por eso empezamos con un for para iterar por cada una de las columnas. 
    for i in range(len(z.columns)): 

        # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
        nombre = "rh_" + str(z[i].apply(pd.Series)["layer"][0]) 

        # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
        valores = list(z[i].apply(pd.Series)["rh"] )

        # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
        df4.insert(i, nombre, valores)

        df4.to_csv("files/5.clima_wind_rh.csv")

    return df4

In [557]:
limpiar_columna_rh ("files/4.clima_wind.csv")

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,14,10,7,8,6,-1,-3,-2,-4,-4,...,25,2,11,1011,none,0,0,115,3,Papua New Guinea
1,14,14,7,8,5,-1,-3,-3,-4,-4,...,26,2,11,1013,none,1,0,100,3,Papua New Guinea
2,14,12,6,6,2,-2,-3,-3,-4,-4,...,26,2,11,1011,none,1,0,90,3,Papua New Guinea
3,14,12,6,6,1,-3,-2,-3,-4,-4,...,26,-1,11,1008,none,1,0,85,3,Papua New Guinea
4,14,13,8,6,1,-3,-2,-3,-4,-4,...,26,2,11,1008,none,1,0,85,3,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,14,13,12,11,2,0,6,4,3,4,...,26,-1,11,1007,rain,3,0,75,3,Papua New Guinea
60,14,13,10,9,3,0,6,3,3,5,...,26,-4,11,1008,rain,3,0,75,3,Papua New Guinea
61,14,13,10,8,4,0,5,3,4,5,...,26,-1,11,1010,rain,3,0,75,4,Papua New Guinea
62,14,12,13,4,2,1,3,5,6,7,...,26,-1,11,1010,rain,3,0,80,3,Papua New Guinea
